## Packaging

In [1]:
import os
import shutil
import numpy as np
import scipy.linalg as la

In [2]:
path = "./fastica"
if os.path.exists(path):
    shutil.rmtree(path)
os.makedirs(path)

In [3]:
% cd fastica/

/Users/Ronnie/Desktop/fastica


In [4]:
%%file fastica.py

import numpy as np
import scipy.linalg as la

# whiten
def whiten(X, n_components):
    X = X - X.mean(-1)[:, None]
    u, d, _ = la.svd(X, full_matrices=False)
    V = (u / d).T[:n_components]
    X *= np.sqrt(X.shape[1])    
    return np.dot(V, X), V

# two wise choices of G
def _logcosh(x):
    # g
    gx = np.tanh(x)
    # g'
    g_x = 1 - gx ** 2
    return gx, g_x.mean(-1)

def _exp(x):
    exp = np.exp(-(x ** 2) / 2)
    # g
    gx = x * exp
    # g'
    g_x = (1 - x ** 2) * exp
    return gx, g_x.mean(axis=-1)

def decorrelation(W):
    s, u = la.eigh(np.dot(W, W.T))
    return np.dot(np.dot(u * (1. / np.sqrt(s)), u.T), W)

# fastICA
def fastICA(X, fun='logcosh', n_components=None, maxIter=200, tol=1e-04):
    X = X.T
    n, m = X.shape
    if n_components is None:
        n_components = n
    X1,V = whiten(X,n_components)
    p = float(m)
    if fun == 'logcosh':
        g = _logcosh
    elif fun == 'exp':
        g = _exp
    
    # initialize w with normal distribution
    W = np.asarray(np.random.normal(size=(n_components,n_components)))

    # calculate w iteratively
    for ii in range(maxIter):
        gwtx, g_wtx = g(np.dot(W, X1))
        W1 = decorrelation(np.dot(gwtx, X1.T) / p - g_wtx[:, None] * W)
        lim = max(abs(abs(np.diag(np.dot(W1, W.T))) - 1))
        W = W1
        if lim < tol:
            break
    S = np.dot(np.dot(W, V), X).T
    return V,W,S

if __name__ == '__main__':
    fastICA()

Writing fastica.py


In [5]:
%%file setup.py

from setuptools import setup
import setuptools

setup(name = "fastica",
      version = "2.0",
      author='Zhechang Yang and Xi Chen',
      author_email='zhechang.yang@duke.edu',
      url='http://people.duke.edu/~ccc14/sta-663-2018/',
      py_modules = ['fastica'],
      packages=setuptools.find_packages(),
      scripts = ['fastica.py'],
      python_requires='>=3',
      )

Writing setup.py


In [6]:
%%file README.md

fastica algorithm

Writing README.md


In [7]:
%%file __init__.py
 
__author__ = 'zhechang'  
__all__=["fastica"]

Writing __init__.py


In [8]:
%%bash

python setup.py install

running install
running bdist_egg
running egg_info
creating fastica.egg-info
writing fastica.egg-info/PKG-INFO
writing dependency_links to fastica.egg-info/dependency_links.txt
writing top-level names to fastica.egg-info/top_level.txt
writing manifest file 'fastica.egg-info/SOURCES.txt'
reading manifest file 'fastica.egg-info/SOURCES.txt'
writing manifest file 'fastica.egg-info/SOURCES.txt'
installing library code to build/bdist.macosx-10.7-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
copying fastica.py -> build/lib
creating build/bdist.macosx-10.7-x86_64
creating build/bdist.macosx-10.7-x86_64/egg
copying build/lib/fastica.py -> build/bdist.macosx-10.7-x86_64/egg
byte-compiling build/bdist.macosx-10.7-x86_64/egg/fastica.py to fastica.cpython-36.pyc
creating build/bdist.macosx-10.7-x86_64/egg/EGG-INFO
installing scripts to build/bdist.macosx-10.7-x86_64/egg/EGG-INFO/scripts
running install_scripts
running build_scripts
creating build/scripts-3.6
cop

zip_safe flag not set; analyzing archive contents...


In [9]:
import fastica as fa

In [10]:
A = np.random.random((4,4))
fa.fastICA(A)

(array([[-4.07186935e-01, -2.60613880e-01, -5.18270463e-01,
          1.01732704e+00],
        [-5.70440410e-02, -2.18594719e+00, -1.35480204e+00,
         -1.27301212e+00],
        [-8.65616215e+00,  4.79568072e+00, -3.56542795e+00,
         -4.05249344e+00],
        [ 2.05632327e+16,  1.65118442e+16, -2.65204800e+16,
         -1.05028759e+15]]),
 array([[ 0.47483926,  0.41145737, -0.76359435, -0.14884279],
        [ 0.2799406 ,  0.62223822,  0.38867602,  0.61917994],
        [ 0.23130399,  0.35504359,  0.48434519, -0.76540986],
        [ 0.8016604 , -0.56344164,  0.17681727,  0.09278885]]),
 array([[-1.95828542e+15,  8.14638742e+15, -1.00702959e+16,
          1.22079848e+15],
        [-1.95828542e+15,  8.14638742e+15, -1.00702959e+16,
          1.22079848e+15],
        [-1.95828542e+15,  8.14638742e+15, -1.00702959e+16,
          1.22079848e+15],
        [-1.95828542e+15,  8.14638742e+15, -1.00702959e+16,
          1.22079848e+15]]))